In [ ]:
#for evaluate

In [ ]:
'''
실험은 여기서 진행하면 될거야
원래 run 함수 만들어서 한방에 돌아가도록 하려했는데 사용자한테 threshold 받아오려면 오히려 비효율적일것 같더라고
최종 목적은 웹이나 시각화 툴 혹은 python 시각화 라이브러리 이용하는 거니까 우선은 이 파일로 실험 진행하고 나중에 프로그램적으로 돌려야겠다 싶으면
그 때 만들어도 될 것 같다.

필요할 것 같은 함수들은 만들어 뒀는데 추가로 필수적인 함수 있으면 구현해서 사용하면 되고 기호에 맞게 수정해도 돼
그리고 함수 추가하거나 수정한 후에 PR 할때는 꼭 말해주고
'''

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cd [your src path]
%cd /content/drive/MyDrive/bistelligence/BISTelligence/src

/content/drive/MyDrive/bistelligence/BISTelligence/src


In [ ]:
%pip install pyod tensorflow

In [4]:
from sklearn.utils.validation import check_is_fitted
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib
     
import data.preprocessing as dp
import model.models as mm

In [6]:
path = '/content/drive/MyDrive/bistelligence/BIS_data/BISTelligen_Project_Data.csv'
train_data, test_data = 0, 0
threshold = 0

trainer = mm.ModelTrain()

In [ ]:
def SetData(scaled=True):
  global train_data, test_data

  train_data, test_data = dp.GetPreprocessedData(path, scaled)

In [ ]:
def GetModel(model_name='MCD', param_dict={}):
  '''
  [default value in 'ModelTrain' Class]
  param_dict={'contamination':0.01,
                     'nu':0.01,
                     'novelty':True,
                     'random_state':42,
                     'n_conponents':1,
                     'cv_type':'full',
                     'momentum':0.9,
                     'learning_rate':0.03,
                     'epochs':100,
                     'patience':10}
  '''

  global trainer

  trainer.SetTrainer(train_data)
  trainer.SetParam(param_dict)
  model = trainer.GetTrainedModel(model_name)

  return model

In [ ]:
def GetAnomalyScore(model=None):
  '''
  모델의 anomaly score return

  나중에 웹이나 시각화 툴로 health index 나타낼 때, (x,y)값 입력해야하는 경우 필요할 것 같아서 만듦.
  혹시 필요 없더라도 문제 없으니까 구현
  '''

  if check_is_fitted(model) == None:
    model_name = type(model).__name__

  if model_name == 'MCD':
    anomaly_score = model.decision_function(test_data)

  elif model_name in ['LocalOutlierFactor','GaussianMixture']:
    anomaly_score = -model.score_samples(test_data)

  elif model_name == 'OneClassSVM':
    anomaly_score = np.max(model.decision_function(test_data))-(model.decision_function(test_data))

  elif model_name == 'IsolationForest':
    anomaly_score = np.max(model.score_samples(test_data))-(model.score_samples(test_data))

  else:
    reconstruction = model.predict(test_data)
    mse = np.mean(np.power(test_data-reconstruction,2),axis=1)
    anomaly_score = ss.zscore(mse)

  return anomaly_score

In [ ]:
def ShowHealthIndex(model = None, anomaly_score = None, scaled = True):
  '''
  GetAnomalyScore로 이상치 점수 받아온 후, health index plot을 도출
  png로 저장 혹은 단순 plot 도출
  '''
  plt.rcParams['figure.figsize'] = (12, 8)
  model_name = type(model).__name__
  sns.scatterplot(range(len(anomaly_score)), anomaly_score)
  plt.title('{} health index (scaled ={}) '.format(model_name, scaled), fontsize = 25)
  plt.show()

In [ ]:
def DoXAI(model=None):
  '''
  더 잘맞는 함수 이름으로 바꿔도 괜찮고, 파라미터도 상황에 맞게 변경 가능
  (threshold를 입력받아야 XAI가 진행될 수도 있다고 생각되는데 그럴 경우, SetThreshold() 함수를 만들어서 threshold에 값 저장시킬 필요 있음.)
  XAI 폴더의 xai.py(가명)의 함수들을 실행시키는 함수
  
  xai.py(가명)의 함수들은 shap에서 지원하는 여러가지 plot을 그리고, png 파일로 XAI 폴더 내에 저장하는 동작으로 가정
  따라서 DoXAI()를 실행하면 XAI 폴더 내에 여러 plot png파일이 생성되는것을 생각하고 만듦
  '''  
  pass

In [ ]:
def SaveBestModel(model):
  # model.saved_model 폴더에 모델 저장
  %cd /content/drive/MyDrive/bistelligence/BISTelligence/src/model/saved_model
  filename = '_'.join(['best', type(model).__name__])
  joblib.dump(model, filename)

In [ ]:
def LoadBestModel(model):
  %cd /content/drive/MyDrive/bistelligence/BISTelligence/src/model/saved_model
  filename = '_'.join(['best', type(model).__name__])
  model = joblib.load(filename)
  return model